In [ ]:
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install ipywidgets

In [ ]:
import json
from datasets import load_dataset, load_metric, load_from_disk
import pandas as pd
from transformers import T5Model, T5ForConditionalGeneration, T5Tokenizer
from transformers import Adafactor
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
class DataClass:

  def __init__(self, data_dir):
    self.data_dir = data_dir

  def data_csv(self, f, output):

    contxtqs = []
    rewrites = []

    filepath = self.data_dir+f

    with open(filepath) as fl:
      data = json.load(fl)
      
      for d in data:
        contxtq = d['context']+' '+d['question']  # concatenating context and contextual question
        contxtqs.append(contxtq)
        rewrites.append(d['rewrite'])

      data = {'conv':contxtqs, 'rewrite':rewrites}
      df = pd.DataFrame(data)
      df.to_csv(output, index=False)


data = DataClass('/storage/qrecc/')

data.data_csv('qrecc_train.json', 'train.csv')
data.data_csv('qrecc_test.json', 'test.csv')

qrecc = load_dataset('csv', data_files={'train': 'train.csv', 'test': 'test.csv'})

In [ ]:
max_length= 384
batch_size = 4
dim = 768 # change BERT hidden size to change
weight_decay = 0.01

pretrained_model = 't5-base'

tokenizer = T5Tokenizer.from_pretrained(pretrained_model)
model = T5ForConditionalGeneration.from_pretrained(pretrained_model)

In [ ]:
def tokenize_dataset(batch):
  context = tokenizer(batch['conv'], padding='max_length', truncation=True, max_length=max_length)
  rewrites = tokenizer(batch['rewrite'], padding='max_length', truncation=True, max_length=max_length)

  batch['ctx_input_ids'] = context.input_ids
  batch['rwrt_input_ids'] = rewrites.input_ids
 

  batch['ctx_attention_mask'] = context.attention_mask
  batch['rwrt_attention_mask'] = rewrites.attention_mask

  return batch


# removing examples with no context
qrecc = qrecc.filter(lambda x: isinstance(x['conv'], str) and isinstance(x['rewrite'], str))

# removing examples with context length > 384
qrecc = qrecc.filter(lambda x: len(tokenizer(x['conv']).input_ids) <= max_length)


# tokenizing
dataset = qrecc.map(
    tokenize_dataset, 
    batch_size = batch_size,
    batched=True,
    remove_columns=['conv', 'rewrite']
)


dataset.set_format(
    type='torch', columns=['ctx_input_ids', 'rwrt_input_ids', 'ctx_attention_mask', 'rwrt_attention_mask'],)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset['train'], batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(dataset['test'], batch_size=batch_size)

In [ ]:
def valid_loss():
  
  val_loss = 0
  idx = 0

  for batch in test_loader:

    ctx_input = batch['ctx_input_ids'].to(device) # QR input
    ctx_attention = batch['ctx_attention_mask'].to(device)

    rwrt_input = batch['rwrt_input_ids'].to(device) 
    rwrt_input[rwrt_input == tokenizer.pad_token_id] = -100
    rwrt_input = rwrt_input.to(device)

    loss = model(input_ids=ctx_input, attention_mask=ctx_attention, labels=rwrt_input).loss
    val_loss += loss.item()

    del ctx_input, ctx_attention, rwrt_input, loss

    idx += 1

  return val_loss/idx

In [ ]:
num_epochs = 3

device = torch.device('cuda')
model.to(device)

#ctx_encoder.load_state_dict(torch.load('/content/ctx_encoder5.pth'))

model.train()

optim = optimizer = Adafactor(
    model.parameters(),
    lr=1e-5,
    eps=(1e-30, 1e-3),
    clip_threshold=1.0,
    decay_rate=-0.8,
    beta1=None,
    weight_decay=0.0,
    relative_step=False,
    scale_parameter=False,
    warmup_init=False
)

for epoch in range(num_epochs):
  
  epoch_loss = 0

  for batch in train_loader:

    ctx_input = batch['ctx_input_ids'].to(device) # QR input
    ctx_attention = batch['ctx_attention_mask'].to(device)

    rwrt_input = batch['rwrt_input_ids'].to(device) 
    rwrt_input[rwrt_input[:, :] == tokenizer.pad_token_id] = -100 # tokens with indices set to -100 are ignored (masked)
    rwrt_input = rwrt_input.to(device)

    loss = model(input_ids=ctx_input, attention_mask=ctx_attention, labels=rwrt_input).loss
    epoch_loss += loss.item() 

    loss.backward()
    optim.step()
    optim.zero_grad()


    del ctx_input, ctx_attention, rwrt_input, loss

  print('Train loss after epoch {} : {}'.format(epoch+1, epoch_loss/len(train_loader)))
  model.eval()
  print('Valid loss after epoch {} : {}'.format(epoch+1, valid_loss()))
  print('\n')
  model.train()
  torch.save(model.state_dict(), 'qr_gen'+str(epoch+1)+'.pth')